# Import Library

In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout


In [4]:
filePath = 'dataset/'

fileName = []
tag = []
full_path = []
labels = []
for path, subdirs, files in os.walk(filePath):
    for name in files:
        full_path.append(os.path.join(path, name))
        tag.append(path.split('/')[-1])
        fileName.append(name)

In [6]:
dataset_path = 'dataset/'

# Lists to store data
file_names = []
labels = []
full_paths = []

# Traverse the dataset directory
for class_folder in sorted(os.listdir(dataset_path)):
    class_folder_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_folder_path):  # Check if it's a directory
        for image_file in os.listdir(class_folder_path):
            full_paths.append(os.path.join(class_folder_path, image_file))
            file_names.append(image_file)
            labels.append(class_folder)  # Use folder name as class label

# Create a DataFrame
data = {
    'FileName': file_names,
    'ClassLabel': labels,
    'FullPath': full_paths
}
df = pd.DataFrame({"FileName": file_names, 'ClassLabel':labels, 'FullPath':full_path})
df.groupby(['ClassLabel']).size()


ClassLabel
ayam           600
brokoli        600
ikan           600
kambing        600
kembang kol    600
kentang        600
kol            600
labu           600
mentimun       600
paprika        600
sapi           600
tahu           600
telur          600
tempe          600
tomat          600
udang          600
wortel         600
dtype: int64

In [7]:
df.head()

,FileName,ClassLabel,FullPath
0,ayam0.jpg,ayam,dataset/ayam\ayam0.jpg
1,ayam1.jpg,ayam,dataset/ayam\ayam1.jpg
2,ayam10.jpg,ayam,dataset/ayam\ayam10.jpg
3,ayam100.jpg,ayam,dataset/ayam\ayam100.jpg
4,ayam101.png,ayam,dataset/ayam\ayam101.png


In [30]:
x = df['FullPath'].values
y = df['ClassLabel'].values

# x adalah gambar dan y adalah label
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=300)

# Pertama, bagi data menjadi train dan temp (yang akan dibagi lagi menjadi validasi dan testing)
for train_index, temp_index in sss.split(x, y):
    X_train, X_temp = x[train_index], x[temp_index]
    y_train, y_temp = y[train_index], y[temp_index]

# Kedua, bagi temp menjadi validation dan testing
sss_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=300)
for val_index, test_index in sss_temp.split(X_temp, y_temp):
    X_val, X_test = X_temp[val_index], X_temp[test_index]
    y_val, y_test = y_temp[val_index], y_temp[test_index]


In [31]:
print(f"Training size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")

Training size: 7140
Validation size: 1530
Test size: 1530


In [41]:
DIMENSION = (180, 180, 3)
BATCH_SIZE = 32
EPOCH = 15
NUM_CLASSES = 17

# Image Preprocessing

In [10]:
#apply laplacian filter
def PreProcess(path):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_GRAY2RGB)
    blurred = cv2.GaussianBlur(img_rgb, (3, 3), 0)
    laplace = cv2.Laplacian(blurred, cv2.CV_64F)
    laplace_abs = cv2.convertScaleAbs(laplace)
    sharpened = cv2.convertScaleAbs(img_rgb - laplace_abs)

    plt.subplots(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(img_rgb)
    plt.title("Original")

    plt.subplot(1,2,2)
    plt.imshow(sharpened)
    plt.title("Sharpened")

    plt.show()



In [ ]:
def PreProcess1(path):
    img = cv2.imread(path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
    laplace = cv2.Laplacian(img_rgb, cv2.CV_64F, ksize=3)
    laplace_abs = cv2.convertScaleAbs(laplace)

    sharpened = cv2.convertScaleAbs(img_rgb - laplace_abs)

    
    # Display results
    plt.subplots(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(img_rgb, 0)
    plt.title("Original")
    
    plt.subplot(1,2,2)
    plt.imshow(sharpened, 0)
    plt.title("Sharpened")
    
    plt.show()
    
    return sharpened

In [36]:
def ImagePreprocess1(path):
    # Load the image 
    image = cv2.imread(path) 
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = cv2.resize(img_rgb, (180,180))

    # Create the sharpening kernel 
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) 

    # Sharpen the image 
    sharpened_image = cv2.filter2D(img_resize, -1, kernel) 
    return sharpened_image

In [42]:
preprocessed_img_train = [ImagePreprocess1(path) for path in X_train]
X_train_preprocessed = np.array(preprocessed_img_train, dtype=np.float32)

preprocessed_img_val = [ImagePreprocess1(path) for path in X_val]
X_val_preprocessed = np.array(preprocessed_img_val, dtype=np.float32)

preprocessed_img_test = [ImagePreprocess1(path) for path in X_test]
X_test_preprocessed = np.array(preprocessed_img_test, dtype=np.float32)

In [43]:
def changeLabel(name):
    if name == "ayam":
        return 0
    elif name == "brokoli":
        return 1
    elif name == "ikan":
        return 2
    elif name == "kambing":
        return 3
    elif name == "kembang kol":
        return 4
    elif name == "kentang":
        return 5
    elif name == "kol":
        return 6
    elif name == "labu":
        return 7
    elif name == "mentimun":
        return 8
    elif name == "paprika":
        return 9
    elif name == "sapi":
        return 10
    elif name == "tahu":
        return 11
    elif name == "telur":
        return 12
    elif name == "tempe":
        return 13
    elif name == "tomat":
        return 14
    elif name == "udang":
        return 15
    elif name == "wortel":
        return 16


In [44]:
num_labels = np.array([changeLabel(i) for i in y_train], dtype=np.float32)
val_labels = np.array([changeLabel(j) for j in y_val], dtype=np.float32)
test_labels = np.array([changeLabel(k) for k in y_test], dtype=np.float32)
y_train_one_hot = to_categorical(num_labels, num_classes=17)
y_val_one_hot = to_categorical(val_labels, num_classes=17)
y_test_one_hot = to_categorical(test_labels, num_classes=17)

In [52]:
validation_img = (np.array(preprocessed_img_val), y_val_one_hot)

# Train into model  

In [46]:
model = models.Sequential([
    tf.keras.Input(shape=(DIMENSION)),  # Input layer
    tf.keras.layers.Rescaling(1./255),  # Normalize the image
    layers.Conv2D(64, (3, 3), activation='relu'),  # First convolutional layer
    layers.MaxPooling2D((2, 2)),  # MaxPooling layer
    layers.Conv2D(32, (3, 3), activation='relu'),  # Second convolutional layer
    layers.Flatten(),  # Flatten the output for dense layers
    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),  # Fully connected layer
    layers.Dense(17, activation='softmax')  # Output layer (17 classes)
])


In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 178, 178, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 89, 89, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 87, 87, 32)        18464     
                                                                 
 flatten_2 (Flatten)         (None, 242208)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                15501376  
                                                      

In [48]:
print(X_train_preprocessed.shape)
print(X_test_preprocessed.shape)


(7140, 180, 180, 3)
(1530, 180, 180, 3)


In [49]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
try:
	model.evaluate(X_train_preprocessed, y_train_one_hot, verbose=False)
except:
	print("Your model is not compatible with the dataset you defined earlier. Check that the loss function, last layer and label_mode are compatible with one another.")
else:
	predictions = model.predict(X_train_preprocessed, verbose=False)
	print(f"predictions have shape: {predictions.shape}")

predictions have shape: (7140, 17)


In [51]:
print(f'Input shape: {model.input_shape}')
print(f'Output shape: {model.output_shape}')

Input shape: (None, 180, 180, 3)
Output shape: (None, 17)


In [53]:
features = np.array(validation_img[0])  # Convert features list to a NumPy array
labels = np.array(validation_img[1])    # Convert labels list to a NumPy array

print(features.shape)  # Features shape
print(labels.shape)    # Labels shape

(1530, 180, 180, 3)
(1530, 17)


In [54]:
history = model.fit(X_train_preprocessed, y_train_one_hot, epochs=EPOCH, batch_size=BATCH_SIZE, validation_data=validation_img)

Epoch 1/15
224/224 [==============================] - 115s 511ms/step - loss: 3.2137 - accuracy: 0.1916 - val_loss: 2.5392 - val_accuracy: 0.2922
Epoch 2/15
224/224 [==============================] - 110s 493ms/step - loss: 2.5469 - accuracy: 0.3176 - val_loss: 2.3614 - val_accuracy: 0.3745
Epoch 3/15
224/224 [==============================] - 112s 501ms/step - loss: 2.2840 - accuracy: 0.4115 - val_loss: 2.2316 - val_accuracy: 0.3804
Epoch 4/15
224/224 [==============================] - 110s 492ms/step - loss: 2.1385 - accuracy: 0.4437 - val_loss: 2.4793 - val_accuracy: 0.4118
Epoch 5/15
224/224 [==============================] - 112s 499ms/step - loss: 2.0338 - accuracy: 0.4769 - val_loss: 1.9486 - val_accuracy: 0.4843
Epoch 6/15
224/224 [==============================] - 111s 497ms/step - loss: 1.9235 - accuracy: 0.5050 - val_loss: 2.0075 - val_accuracy: 0.4732
Epoch 7/15
224/224 [==============================] - 111s 495ms/step - loss: 1.8639 - accuracy: 0.5282 - val_loss: 1.9649 -

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Training and validation accuracy')

for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Accuracy", "Loss"])):
    ax[i].plot(epochs, data[0], 'r', label="Training " + label)
    ax[i].plot(epochs, data[1], 'b', label="Validation " + label)
    ax[i].legend()
    ax[i].set_xlabel('epochs')

plt.show()